In [1]:
import tensorflow as tf

# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
# Parameters

learning_rate = 0.01
training_epochs = 25
batch_size = 100
display_step = 1
logs_path = '/tmp/tensorflow_logs/example'

x = tf.placeholder(tf.float32, [None, 784], name='InputData')
y = tf.placeholder(tf.float32, [None, 10], name='LabelData')

W = tf.Variable(tf.zeros([784, 10]), name='Weights')
b = tf.Variable(tf.zeros([10]), name='Bias')

In [3]:
# Model

with tf.name_scope('Model'):
    # Model
    pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax
with tf.name_scope('Loss'):
    # Minimize error using cross entropy
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
with tf.name_scope('SGD'):
    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
with tf.name_scope('Accuracy'):
    # Accuracy
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))

In [4]:
# Initializing

init = tf.global_variables_initializer()

# Create a summary to monitor cost tensor
tf.summary.scalar("loss", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("accuracy", acc)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

In [5]:
# Launch

with tf.Session() as sess:
    sess.run(init)

    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop), cost op (to get loss value)
            # and summary nodes
            _, c, summary = sess.run([optimizer, cost, merged_summary_op], feed_dict={x: batch_xs, y: batch_ys})
            # Write logs at every iteration
            summary_writer.add_summary(summary, epoch * total_batch + i)
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print "Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost)

    print "Optimization Finished!"

    # Test model
    # Calculate accuracy
    print "Accuracy:", acc.eval({x: mnist.test.images, y: mnist.test.labels})

    print "Run the command line:\n" \
          "--> tensorboard --logdir=/tmp/tensorflow_logs " \
          "\nThen open http://0.0.0.0:6006/ into your web browser"

Epoch: 0001 cost= 1.182138980
Epoch: 0002 cost= 0.664788804
Epoch: 0003 cost= 0.552656525
Epoch: 0004 cost= 0.498552776
Epoch: 0005 cost= 0.465371131
Epoch: 0006 cost= 0.442519504
Epoch: 0007 cost= 0.425472137
Epoch: 0008 cost= 0.412135788
Epoch: 0009 cost= 0.401333047
Epoch: 0010 cost= 0.392321333
Epoch: 0011 cost= 0.384786582
Epoch: 0012 cost= 0.378121959
Epoch: 0013 cost= 0.372402035
Epoch: 0014 cost= 0.367300448
Epoch: 0015 cost= 0.362672446
Epoch: 0016 cost= 0.358639969
Epoch: 0017 cost= 0.354909423
Epoch: 0018 cost= 0.351425165
Epoch: 0019 cost= 0.348299438
Epoch: 0020 cost= 0.345420638
Epoch: 0021 cost= 0.342725528
Epoch: 0022 cost= 0.340243397
Epoch: 0023 cost= 0.337939342
Epoch: 0024 cost= 0.335723945
Epoch: 0025 cost= 0.333761451
Optimization Finished!
Accuracy: 0.9132
Run the command line:
--> tensorboard --logdir=/tmp/tensorflow_logs 
Then open http://0.0.0.0:6006/ into your web browser
